# Pull ESPN Ownership Data

### MSP 9.4.2017


In [1]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# boilerplate imports

import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot




In [3]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator



### Change In Ownership

Also set up an archive.

In [9]:

Ownership = {}


for tablenum in range(0,13):

    # cycle through all positions
    r  = requests.get('http://games.espn.com/flb/addeddropped?slotCategoryId='+str(tablenum))

    data = r.text

    soup = BeautifulSoup(data)

    tables = soup.findAll('table')


    for table in tables: 

        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        #print headings


        for row in table.find_all("tr")[1:]: 

        #    sav = [td.find('a') for td in row.find_all("td")]
            sav = [td.get_text() for td in row.find_all("td")]

            #print(sav)
            try:
                w = sav.index(u'\xa0')
            except:
                w = 0
                
            #print(w)

            #print (sav[1].split(',')[0]).strip('*')

            try:
                # block out all non-rankings
                xx = float(sav[0])

                #print sav[0:w]
                #print sav[w+1:len(sav)]

                # added
                print((sav[1].split(',')[0]).strip('*'))
                Ownership[(sav[1].split(',')[0]).strip('*')] = float(sav[5])

                #dropped
                Ownership[(sav[w+2].split(',')[0]).strip('*')] = float(sav[w+6])

            except:
                pass

            #[u'RK', u'PLAYER, TEAM', u'POS', u'LAST', u'CUR', u'7-DAY +/-']
        

today = datetime.date.today()

#f = open('ownership-archive/'+str(today)+'.txt','w')

#for index,key in enumerate(Ownership.keys()):
#    print >>f,key,Ownership[key]
    
#f.close()

Yasmani Grandal
Brian McCann
Jonathan Lucroy
Welington Castillo
Chris Iannetta
Christian Vazquez
Wilson Ramos
Austin Hedges
Robinson Chirinos
Russell Martin
Martin Maldonado
James McCann
Tucker Barnhart
Austin Barnes
Kevin Plawecki
Travis d'Arnaud
Yadier Molina
Drew Butera
Matt Wieters
Kurt Suzuki
Francisco Cervelli
Jorge Alfaro
Manny Pina
Alex Avila
Yan Gomes
Yasmani Grandal
Brian McCann
Jonathan Lucroy
Welington Castillo
Chris Iannetta
Christian Vazquez
Wilson Ramos
Austin Hedges
Robinson Chirinos
Russell Martin
Martin Maldonado
James McCann
Tucker Barnhart
Austin Barnes
Kevin Plawecki
Travis d'Arnaud
Yadier Molina
Drew Butera
Matt Wieters
Kurt Suzuki
Francisco Cervelli
Jorge Alfaro
Manny Pina
Alex Avila
Yan Gomes
Jose Martinez
Lucas Duda
Ryon Healy
Joe Mauer
Adrian Gonzalez
Logan Morrison
Brandon Belt
Tyler Austin
Daniel Vogelbach
Chase Headley
Colin Moran
Luis Valbuena
Ryan Rua
Andrew Romine
Garrett Cooper
Danny Valencia
Ryan McMahon
Jose Martinez
Lucas Duda
Ryon Healy
Joe Mauer
Ad

In [ ]:
print l[0]

positives = ['congratulates']

print np.sum(np.array([(x in l[0]) for x in positives]))



In [ ]:
negatives = ['rehab','injure','disable','DL','suspension','surgery','injury']

#[(x in string_in) for x in negatives]

def check_words(string_in,words):
    total = [(x in string_in) for x in words]
    return np.sum(np.array(total))
    

for index,key in enumerate(Ownership.keys()):
    if Ownership[key] < -5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print check_words(l[0],negatives),' ',
        print check_words(l[1],negatives),' ',
        print check_words(l[2],negatives)
        print ''



In [ ]:
for index,key in enumerate(Ownership.keys()):
    if Ownership[key] > 5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print ''




In [ ]:
print Ownership['Justin Verlander']

### News Alerts

Another interesting thing is grabbing Google News findings on players to see if we can determine what the narrative leading to the explosion in pickups or drops is coming from.

Good keywords:
1. Trade
2. Hot
3. Streak
4. Fire
5. Homer


Bad keywords:
1. Cold
2. Bench
3. Disabled/DL
4. Injure
5. Suspension



In [ ]:
#https://stackoverflow.com/questions/39354587/scraping-google-news-with-beautifulsoup-returns-empty-results

from bs4 import BeautifulSoup
import requests
import time
from random import randint


def scrape_news_summaries(s):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    
    r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")    
    # Print the status code
    #print(r.status_code)  
    
    content = r.text
    #print content
    
    news_summaries = []
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.find_all("div", class_="gs-info")
    st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("div", {"class": "rc"})


    for st_div in st_divs:
        #print st_div
        news_summaries.append(st_div.text)
    return news_summaries




#http://www.espn.com/search/results?q=Alex+Wood
#r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")

l = scrape_news_summaries("Justin+Verlander")

for n in l:
    print(n)
    print('')

### Stats Alerts

We should also scrape the fangraphs gamelog page to see if there is anything obvious there.

### Crazy

It would be awesome to try and make a model to predict ownership data based on news stories.

In [ ]:
r  = requests.get('http://www.espn.com/mlb/player/_/id/33095/jose-abreu')

data = r.text

soup = BeautifulSoup(data)

fantasy_table = soup.find_all('div', id='fantasy-content')

print fantasy_table#.find("tr")

#for table in tables: 

#        headings = [th.get_text() for th in table.find("tr").find_all("th")]
#        #print headings

#<div class="mod-content" id="fantasy-content">


#(row in table.find_all("tr", class_="grid_postseason"))